Paso 1: Obtención de Datos
Encontrar o construir un dataset que tenga las siguientes características:
•	Que tenga al menos 5 columnas que formen parte del conjunto de variables independientes. Que entre dichas columnas haya al menos:
o	3 columnas con datos numéricos. Al menos una de estas columnas debe tener datos ausentes
o	1 columna con datos categóricos de tipo nominal
•	Que tenga una columna que sea la variable dependiente (variable objetivo o etiqueta)


In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import time
# Semilla para reproducibilidad
np.random.seed(42)

# Crear el dataset
data = {
    'Edad': np.random.randint(18, 66, size=1000),  
    'Ingreso anual': np.random.randint(25000, 100000, size=1000),
    'Horas de estudio por semana': np.random.choice([np.nan, 20, 30, 40], size=1000),
    'Género': np.random.choice(['Masculino', 'Femenino'], size=1000),
    'Región': np.random.choice(['Norte', 'Sur', 'Este', 'Oeste'], size=1000),
    'Sueldo mensual': np.random.randint(2000, 8000, size=1000)  # Variable dependiente
}

df = pd.DataFrame(data)

# Mostrar las primeras filas del DataFrame
df.head()


# Dividir en variables independientes (X) y dependiente (y)
X = df.drop('Sueldo mensual', axis=1)  # Variables independientes
y = df['Sueldo mensual']  # Variable dependiente

# Separar las variables numéricas y categóricas
X_num = X.select_dtypes(include=['float64', 'int64'])
X_cat = X.select_dtypes(include=['object'])

# Tratamiento de datos numéricos: imputación de valores ausentes y normalización
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputación de valores ausentes con la media
    ('scaler', StandardScaler())  # Normalización
])

# Tratamiento de datos categóricos: OneHotEncoding
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Codificación OneHot de variables categóricas
])

# Aplicar el preprocesamiento a los datos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, X_num.columns),  # Aplicar el pipeline de datos numéricos
        ('cat', categorical_transformer, X_cat.columns)  # Aplicar el pipeline de datos categóricos
    ])

# Crear el conjunto de entrenamiento y test (75% entrenamiento, 25% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Preprocesar los datos de entrenamiento y prueba
start_time = time.time()
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)
end_time = time.time()
tiempo_preprocesamiento = end_time - start_time

# Mostrar el tiempo de preprocesamiento
tiempo_preprocesamiento


0.0170896053314209

Paso 2: Preprocesamiento de Datos
Sobre dicho dataset se deberán llevar a cabo los siguientes tratamientos de preprocesado de datos:
•	Gestión de datos ausentes: se deberán tratar los datos ausentes de una forma razonable en términos del negocio que representen dichos datos y que en ningún caso sea eliminando las filas y/o columnas que contengan datos ausentes
•	Gestión de datos categóricos de tipo nominal: se tratarán a través de la técnica de One Hot Encoding
•	Se utilizarán el 75% de los datos para construir el modelo y el 25% restante para validar el modelo construido
•	Se normalizarán o estandarizarán (a elegir por el alumno) todos los datos asociados de las variables independientes
El preprocesamiento deberá realizarse una única vez, y no una vez por cada modelo generado.
Al final se deberá deberá mostrar el tiempode ejecución gastado del preprocesamiento de los datos.


In [38]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, max_error, r2_score

# Función para entrenar y evaluar modelos de regresión lineal
def evaluar_modelo_lineal(X_train, X_test, y_train, y_test, columnas_utilizadas):
    regressor = LinearRegression()
    
    # Entrenar el modelo con las columnas seleccionadas
    X_train_sub = X_train[:, columnas_utilizadas]
    X_test_sub = X_test[:, columnas_utilizadas]
    
    start_time = time.time()
    regressor.fit(X_train_sub, y_train)
    end_time = time.time()
    tiempo_entrenamiento = end_time - start_time

    # Realizar las predicciones
    start_time = time.time()
    y_pred = regressor.predict(X_test_sub)
    end_time = time.time()
    tiempo_prueba = end_time - start_time

    # Calcular las métricas
    M = max_error(y_test, y_pred)
    MAE = mean_absolute_error(y_test, y_pred)
    R2 = r2_score(y_test, y_pred)
    
    return M, MAE, R2, tiempo_entrenamiento, tiempo_prueba

# Modelos de regresión lineal (todas las variables, ignorando una y dos variables)
resultados_lineales = []

# Utilizando todas las variables
M, MAE, R2, tiempo_entrenamiento, tiempo_prueba = evaluar_modelo_lineal(X_train, X_test, y_train, y_test, np.arange(X_train.shape[1]))
resultados_lineales.append(['Regresión Lineal', 'Todas las variables', M, MAE, R2, tiempo_entrenamiento, tiempo_prueba])

# Ignorando una variable (por ejemplo, 'Edad')
columnas_sin_edad = np.delete(np.arange(X_train.shape[1]), 0)
M, MAE, R2, tiempo_entrenamiento, tiempo_prueba = evaluar_modelo_lineal(X_train, X_test, y_train, y_test, columnas_sin_edad)
resultados_lineales.append(['Regresión Lineal', 'Sin Edad', M, MAE, R2, tiempo_entrenamiento, tiempo_prueba])

# Ignorando dos variables (por ejemplo, 'Edad' y 'Ingreso anual')
columnas_sin_edad_ingreso = np.delete(columnas_sin_edad, 0)
M, MAE, R2, tiempo_entrenamiento, tiempo_prueba = evaluar_modelo_lineal(X_train, X_test, y_train, y_test, columnas_sin_edad_ingreso)
resultados_lineales.append(['Regresión Lineal', 'Sin Edad e Ingreso Anual', M, MAE, R2, tiempo_entrenamiento, tiempo_prueba])

# Mostrar los resultados
resultados_lineales


[['Regresión Lineal',
  'Todas las variables',
  np.float64(3162.2400318290765),
  1420.8637463078965,
  -0.006673769286622999,
  0.0030372142791748047,
  0.0],
 ['Regresión Lineal',
  'Sin Edad',
  np.float64(3166.8932036248907),
  1422.5588279486608,
  -0.007059189735710181,
  0.0009474754333496094,
  0.0],
 ['Regresión Lineal',
  'Sin Edad e Ingreso Anual',
  np.float64(3166.893203624889),
  1422.5588279486608,
  -0.007059189735709959,
  0.0011219978332519531,
  0.0]]

Paso 3: Generación de Modelos
La práctica consistirá en generar diferentes modelos utilizando las siguientes técnicas de regresión:
•	Regresión Lineal. Se generarán los siguientes modelos:
o	Utilizando todas las variables independientes
o	Ignorando una de las variables independientes (la que en apariencia sea menos relevante)
o	Ignorando dos de las variables independientes (las que en apariencia sean menos relevantes)
•	Regresión Polinómica. Se generarán los siguientes modelos:
o	Utilizando todas las variables independientes y generando un modelo polinómico de grado 12, otro de grado 8 y otro de grado 4
o	Ignorando una de las variables independientes (la que en apariencia sea menos relevante) y generando un modelo polinómico de grado 10, otro de grado 7 y otro de grado 5
o	Ignorando dos de las variables independientes (las que en apariencia sean menos relevantes) y generando un modelo polinómico de grado 8, otro de grado 5 y otro de grado 2
•	Regresión de las K Valores más Cercanos. Se generarán los siguientes modelos:
o	Utilizando todas las variables independientes y generando un modelo con K = 8, otro con K = 6 y otro con K = 4
o	Ignorando una de las variables independientes (la que en apariencia sea menos relevante) y generando un modelo con K = 6, otro con K = 4 y otro con K = 2
o	Ignorando dos de las variables independientes (las que en apariencia sean menos relevantes) y generando un modelo con K =4, otro con K = 2 y otro con K = 1
Obteniendo para cada uno de los modelos las siguientes métricas:
•	Máximo Error Absoluto (M)
•	Error Absoluto Medio (MAE)
•	Coeficiente de Determinación (R^2)
Mostrando en una tabla (que sea una dataframe por ejemplo):
•	Las métricas obtenidas para todos y cada uno de los modelos
•	El tiempo de ejecución gastado en el entrenamiento de cada modelo (excluido el tiempo de preprocesamiento)
•	El tiempo de ejecución gastado en la prueba de cada modelo


In [39]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Función para entrenar y evaluar modelos de regresión polinómica
def evaluar_modelo_polinomico(X_train, X_test, y_train, y_test, columnas_utilizadas, grado):
    poly_regressor = make_pipeline(PolynomialFeatures(degree=grado), LinearRegression())
    
    # Entrenar el modelo con las columnas seleccionadas
    X_train_sub = X_train[:, columnas_utilizadas]
    X_test_sub = X_test[:, columnas_utilizadas]
    
    start_time = time.time()
    poly_regressor.fit(X_train_sub, y_train)
    end_time = time.time()
    tiempo_entrenamiento = end_time - start_time

    # Realizar las predicciones
    start_time = time.time()
    y_pred_poly = poly_regressor.predict(X_test_sub)
    end_time = time.time()
    tiempo_prueba = end_time - start_time

    # Calcular las métricas
    M = max_error(y_test, y_pred_poly)
    MAE = mean_absolute_error(y_test, y_pred_poly)
    R2 = r2_score(y_test, y_pred_poly)
    
    return M, MAE, R2, tiempo_entrenamiento, tiempo_prueba

# Modelos de regresión polinómica (todas las variables, ignorando una y dos variables)
resultados_polinomicos = []

# Utilizando todas las variables
for grado in [12, 8, 4]:
    M, MAE, R2, tiempo_entrenamiento, tiempo_prueba = evaluar_modelo_polinomico(X_train, X_test, y_train, y_test, np.arange(X_train.shape[1]), grado)
    resultados_polinomicos.append([f'Regresión Polinómica (grado {grado})', 'Todas las variables', M, MAE, R2, tiempo_entrenamiento, tiempo_prueba])

# Ignorando una variable (por ejemplo, 'Edad')
for grado in [10, 7, 5]:
    M, MAE, R2, tiempo_entrenamiento, tiempo_prueba = evaluar_modelo_polinomico(X_train, X_test, y_train, y_test, columnas_sin_edad, grado)
    resultados_polinomicos.append([f'Regresión Polinómica (grado {grado})', 'Sin Edad', M, MAE, R2, tiempo_entrenamiento, tiempo_prueba])

# Ignorando dos variables (por ejemplo, 'Edad' y 'Ingreso anual')
for grado in [8, 5, 2]:
    M, MAE, R2, tiempo_entrenamiento, tiempo_prueba = evaluar_modelo_polinomico(X_train, X_test, y_train, y_test, columnas_sin_edad_ingreso, grado)
    resultados_polinomicos.append([f'Regresión Polinómica (grado {grado})', 'Sin Edad e Ingreso Anual', M, MAE, R2, tiempo_entrenamiento, tiempo_prueba])

# Mostrar los resultados
resultados_polinomicos


[['Regresión Polinómica (grado 12)',
  'Todas las variables',
  np.float64(3561.6315789474565),
  1481.8206038731967,
  -0.09407095517599928,
  6.959517955780029,
  0.0806887149810791],
 ['Regresión Polinómica (grado 8)',
  'Todas las variables',
  np.float64(3561.631578947342),
  1481.820603873198,
  -0.09407095517599795,
  1.0061454772949219,
  0.016083240509033203],
 ['Regresión Polinómica (grado 4)',
  'Todas las variables',
  np.float64(3561.0728068228445),
  1480.8762677354798,
  -0.09776120093278573,
  0.05872154235839844,
  0.0020017623901367188],
 ['Regresión Polinómica (grado 10)',
  'Sin Edad',
  np.float64(3126.6938775510207),
  1436.9839309760594,
  -0.015790988828899177,
  1.4195466041564941,
  0.015046119689941406],
 ['Regresión Polinómica (grado 7)',
  'Sin Edad',
  np.float64(3126.6938775510207),
  1436.9839309760594,
  -0.015790988828899177,
  0.463611364364624,
  0.0041217803955078125],
 ['Regresión Polinómica (grado 5)',
  'Sin Edad',
  np.float64(3126.6938775510207

In [40]:
from sklearn.neighbors import KNeighborsRegressor

# Función para entrenar y evaluar modelos K-NN
def evaluar_modelo_knn(X_train, X_test, y_train, y_test, columnas_utilizadas, k):
    knn_regressor = KNeighborsRegressor(n_neighbors=k)
    
    # Entrenar el modelo con las columnas seleccionadas
    X_train_sub = X_train[:, columnas_utilizadas]
    X_test_sub = X_test[:, columnas_utilizadas]
    
    start_time = time.time()
    knn_regressor.fit(X_train_sub, y_train)
    end_time = time.time()
    tiempo_entrenamiento = end_time - start_time

    # Realizar las predicciones
    start_time = time.time()
    y_pred_knn = knn_regressor.predict(X_test_sub)
    end_time = time.time()
    tiempo_prueba = end_time - start_time

    # Calcular las métricas
    M = max_error(y_test, y_pred_knn)
    MAE = mean_absolute_error(y_test, y_pred_knn)
    R2 = r2_score(y_test, y_pred_knn)
    
    return M, MAE, R2, tiempo_entrenamiento, tiempo_prueba

# Modelos de K-NN (todas las variables, ignorando una y dos variables)
resultados_knn = []

# Utilizando todas las variables
for k in [8, 6, 4]:
    M, MAE, R2, tiempo_entrenamiento, tiempo_prueba = evaluar_modelo_knn(X_train, X_test, y_train, y_test, np.arange(X_train.shape[1]), k)
    resultados_knn.append([f'K-NN (k={k})', 'Todas las variables', M, MAE, R2, tiempo_entrenamiento, tiempo_prueba])

# Ignorando una variable (por ejemplo, 'Edad')
for k in [6, 4, 2]:
    M, MAE, R2, tiempo_entrenamiento, tiempo_prueba = evaluar_modelo_knn(X_train, X_test, y_train, y_test, columnas_sin_edad, k)
    resultados_knn.append([f'K-NN (k={k})', 'Sin Edad', M, MAE, R2, tiempo_entrenamiento, tiempo_prueba])

# Ignorando dos variables (por ejemplo, 'Edad' y 'Ingreso anual')
for k in [4, 2, 1]:
    M, MAE, R2, tiempo_entrenamiento, tiempo_prueba = evaluar_modelo_knn(X_train, X_test, y_train, y_test, columnas_sin_edad_ingreso, k)
    resultados_knn.append([f'K-NN (k={k})', 'Sin Edad e Ingreso Anual', M, MAE, R2, tiempo_entrenamiento, tiempo_prueba])

# Mostrar los resultados
resultados_knn


[['K-NN (k=8)',
  'Todas las variables',
  np.float64(4229.375),
  1538.673,
  -0.24142075454335732,
  0.0026383399963378906,
  0.0017366409301757812],
 ['K-NN (k=6)',
  'Todas las variables',
  np.float64(4010.0),
  1556.3013333333336,
  -0.2799877056590361,
  0.0013051033020019531,
  0.002235889434814453],
 ['K-NN (k=4)',
  'Todas las variables',
  np.float64(4237.25),
  1598.953,
  -0.3403703450940101,
  0.0009903907775878906,
  0.0027344226837158203],
 ['K-NN (k=6)',
  'Sin Edad',
  np.float64(3625.833333333333),
  1491.3346666666666,
  -0.09979771844287733,
  0.0009968280792236328,
  0.0010042190551757812],
 ['K-NN (k=4)',
  'Sin Edad',
  np.float64(4069.0),
  1607.044,
  -0.35048333123599607,
  0.0009937286376953125,
  0.0019974708557128906],
 ['K-NN (k=2)',
  'Sin Edad',
  np.float64(4150.5),
  1638.098,
  -0.3799861122949735,
  0.0009818077087402344,
  0.0020270347595214844],
 ['K-NN (k=4)',
  'Sin Edad e Ingreso Anual',
  np.float64(3590.25),
  1532.949,
  -0.187579970370638,


In [43]:
# Unir todos los resultados obtenidos en las celdas anteriores
resultados_totales = resultados_lineales + resultados_polinomicos + resultados_knn

# Crear el DataFrame de resultados
df_resultados = pd.DataFrame(resultados_totales, columns=[
    'Modelo', 'Configuración', 'M', 'MAE', 'R2', 'Tiempo Entrenamiento (s)', 'Tiempo Prueba (s)'
])

# Mostrar los resultados
df_resultados


,Modelo,Configuración,M,MAE,R2,Tiempo Entrenamiento (s),Tiempo Prueba (s)
0,Regresión Lineal,Todas las variables,3162.240032,1420.863746,-0.006674,0.003037,0.000000
1,Regresión Lineal,Sin Edad,3166.893204,1422.558828,-0.007059,0.000947,0.000000
2,Regresión Lineal,Sin Edad e Ingreso Anual,3166.893204,1422.558828,-0.007059,0.001122,0.000000
3,Regresión Polinómica (grado 12),Todas las variables,3561.631579,1481.820604,-0.094071,6.959518,0.080689
4,Regresión Polinómica (grado 8),Todas las variables,3561.631579,1481.820604,-0.094071,1.006145,0.016083
5,Regresión Polinómica (grado 4),Todas las variables,3561.072807,1480.876268,-0.097761,0.058722,0.002002
6,Regresión Polinómica (grado 10),Sin Edad,3126.693878,1436.983931,-0.015791,1.419547,0.015046
7,Regresión Polinómica (grado 7),Sin Edad,3126.693878,1436.983931,-0.015791,0.463611,0.004122
8,Regresión Polinómica (grado 5),Sin Edad,3126.693878,1436.983931,-0.015791,0.112411,0.001972
9,Regresión Polinómica (grado 8),Sin Edad e Ingreso Anual,3126.693878,1436.983931,-0.015791,1.157613,0.002622


In [44]:
# Valoración de los modelos generados
# Se observan las métricas y tiempos de ejecución para evaluar qué modelo es el más adecuado
df_resultados.sort_values(by=['MAE', 'R2'], ascending=[True, False])

,Modelo,Configuración,M,MAE,R2,Tiempo Entrenamiento (s),Tiempo Prueba (s)
0,Regresión Lineal,Todas las variables,3162.240032,1420.863746,-0.006674,0.003037,0.000000
2,Regresión Lineal,Sin Edad e Ingreso Anual,3166.893204,1422.558828,-0.007059,0.001122,0.000000
1,Regresión Lineal,Sin Edad,3166.893204,1422.558828,-0.007059,0.000947,0.000000
6,Regresión Polinómica (grado 10),Sin Edad,3126.693878,1436.983931,-0.015791,1.419547,0.015046
7,Regresión Polinómica (grado 7),Sin Edad,3126.693878,1436.983931,-0.015791,0.463611,0.004122
8,Regresión Polinómica (grado 5),Sin Edad,3126.693878,1436.983931,-0.015791,0.112411,0.001972
9,Regresión Polinómica (grado 8),Sin Edad e Ingreso Anual,3126.693878,1436.983931,-0.015791,1.157613,0.002622
10,Regresión Polinómica (grado 5),Sin Edad e Ingreso Anual,3126.693878,1436.983931,-0.015791,0.040936,0.001008
11,Regresión Polinómica (grado 2),Sin Edad e Ingreso Anual,3126.693878,1436.983931,-0.015791,0.001996,0.000996
5,Regresión Polinómica (grado 4),Todas las variables,3561.072807,1480.876268,-0.097761,0.058722,0.002002
